In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("PROJECT").getOrCreate()

In [0]:
import logging
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("POS_Daily_Transaction_File_Processing").getOrCreate()

In [0]:
# Function to check if mountpoint exists

def mount_exists(mount_point):
    mounts = [mount.mountPoint for mount in dbutils.fs.mounts()]
    return mount_point in mounts


# Define the mount point and storage details

mount_point="/mnt/pos_data_testing"
storage_account_name="pharmacydata2024"
container_name="test"
sas_token="DECBZEPbRnh8pmQGIHIRyH9QpredTuDf6nlHoQOn8pHqb1YBPzBHDrvo1uKIojLYh226lFaNS62B+AStdZxCCw=="


# Only create the mount if it doesn't already exist

if not mount_exists(mount_point):
    print(f"Mount point {mount_point} does not exist,creating mount.")

    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
        mount_point=mount_point,
        extra_configs={"fs.azure.account.key.pharmacydata2024.blob.core.windows.net":sas_token}
    )
    
    print(f"Mount point {mount_point} created successfully.")
else:
    print(f"Mount point {mount_point} already exists. No action taken.")


Mount point /mnt/pos_data_testing already exists. No action taken.


In [0]:
dbutils.fs.ls(mount_point)

[FileInfo(path='dbfs:/mnt/pos_data_testing/POS_Sales_Data_2024_demo_testing1.csv', name='POS_Sales_Data_2024_demo_testing1.csv', size=1054, modificationTime=1727337016000),
 FileInfo(path='dbfs:/mnt/pos_data_testing/test-date-processed-output/', name='test-date-processed-output/', size=0, modificationTime=0)]

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS postest

In [0]:
%sql
CREATE TABLE IF NOT EXISTS project_adb.postest.final_city_kpi_test3 (
  StoreLocation STRING,
  TotalSales DOUBLE,
  TotalTransactions BIGINT,
  UniqueCustomers BIGINT,
  AvgTransactionValue DOUBLE,
  RetainedCustomers BIGINT,
  RetentionRate DOUBLE,
  TopSellingProductCategory STRING,
  TopSellingCategorySales DOUBLE
) 
USING DELTA;


In [0]:
%sql
SELECT * FROM project_adb.postest.final_city_kpi_test3;

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales


In [0]:
%sql
SELECT count(*) AS PREV_COUNT FROM project_adb.postest.final_city_kpi_test3;

PREV_COUNT
0


In [0]:
%sql
DESCRIBE project_adb.postest.final_city_kpi_test3;


col_name,data_type,comment
StoreLocation,string,null
TotalSales,double,null
TotalTransactions,bigint,null
UniqueCustomers,bigint,null
AvgTransactionValue,double,null
RetainedCustomers,bigint,null
RetentionRate,double,null
TopSellingProductCategory,string,null
TopSellingCategorySales,double,null


In [0]:
%sql
INSERT INTO project_adb.postest.final_city_kpi_test3 (
    StoreLocation,
    TotalSales,
    TotalTransactions,
    AvgTransactionValue,
    UniqueCustomers,
    RetainedCustomers,
    RetentionRate,
    TopSellingProductCategory,
    TopSellingCategorySales
) VALUES (
    'Chicago',            -- Example StoreLocation
    0,                  -- TotalSales
    0,                    -- TotalTransactions
    0,                  -- AvgTransactionValue
    0,                    -- UniqueCustomers
    0,                    -- RetainedCustomers
    0,                  -- RetentionRate
    NULL,                 -- TopSellingProductCategory (this can be NULL)
    0                  -- TopSellingCategorySales
);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM project_adb.postest.final_city_kpi_test3

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
Chicago,0.0,0,0,0.0,0,0.0,null,0.0


In [0]:
pos_daily_file="/mnt/pos_data_testing/POS_Sales_Data_2024_demo_testing1.csv"
pos_daily_file_test=spark.read.format("csv").option("header","true").option("inferSchema","true").load(pos_daily_file)
display(pos_daily_file_test)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T00140,S001,San Jose,C002,U021,50,Female,P014,AIDS Treatment (Prescription),Diazepam,5,30.0,04-07-2024 02:33,150.0,50.0,10.0,110.0
T0015,S007,New York City,C003,U037,36,Male,P041,Cardiovascular Diseases (Prescription),Lisinopril,1,238.14,12-07-2024 00:16,238.14,29.24,16.6698,225.5698
T0016,S009,Chicago,C005,U009,41,Male,P005,Antibiotics (Prescription),Ciprofloxacin,4,121.95,23-07-2024 18:28,487.8,10.21,34.146,511.736
T0017,S008,Phoenix,C004,U039,58,Female,P010,Bone Health (OTC & Prescription),Cisplatin,2,94.82,10-06-2024 11:59,189.64,14.22,13.2748,188.6948
T0018,S010,San Jose,C005,U059,36,Female,P027,Eye Health (OTC & Prescription),Vitamin D,2,142.14,24-06-2024 07:49,284.28,37.58,19.8996,266.5996
T0019,S005,Philadelphia,C005,U065,53,Female,P008,Diabetes (Prescription & OTC),Olopatadine,4,203.3,19-07-2024 01:52,813.2,0.01,56.924,870.114


In [0]:
df_pos_daily_file=pos_daily_file_test.dropna()
display(df_pos_daily_file)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T00140,S001,San Jose,C002,U021,50,Female,P014,AIDS Treatment (Prescription),Diazepam,5,30.0,04-07-2024 02:33,150.0,50.0,10.0,110.0
T0015,S007,New York City,C003,U037,36,Male,P041,Cardiovascular Diseases (Prescription),Lisinopril,1,238.14,12-07-2024 00:16,238.14,29.24,16.6698,225.5698
T0016,S009,Chicago,C005,U009,41,Male,P005,Antibiotics (Prescription),Ciprofloxacin,4,121.95,23-07-2024 18:28,487.8,10.21,34.146,511.736
T0017,S008,Phoenix,C004,U039,58,Female,P010,Bone Health (OTC & Prescription),Cisplatin,2,94.82,10-06-2024 11:59,189.64,14.22,13.2748,188.6948
T0018,S010,San Jose,C005,U059,36,Female,P027,Eye Health (OTC & Prescription),Vitamin D,2,142.14,24-06-2024 07:49,284.28,37.58,19.8996,266.5996
T0019,S005,Philadelphia,C005,U065,53,Female,P008,Diabetes (Prescription & OTC),Olopatadine,4,203.3,19-07-2024 01:52,813.2,0.01,56.924,870.114


In [0]:
#Citywise KPI Calculation

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

# Step 1: Convert Timestamp to Date
df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Calculate Total Sales and Number of Transactions by City
citywise_sales = df_with_date.groupBy("StoreLocation") \
    .agg(
        F.sum("TotalAmount").alias("TotalSales"),
        F.count("TransactionID").alias("TotalTransactions"),
        F.countDistinct("CustomerID").alias("UniqueCustomers")
    )

# Step 3: Calculate Average Transaction Value
citywise_sales = citywise_sales.withColumn("AvgTransactionValue", col("TotalSales") / col("TotalTransactions"))

# Step 4: Calculate Retained Customers
customer_city_purchases = df_with_date.groupBy("CustomerID", "StoreLocation") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

retained_customers_city = customer_city_purchases.groupBy("StoreLocation") \
    .agg(F.count(F.when(col("TotalPurchases") > 1, 1)).alias("RetainedCustomers"))

# Step 5: Join the retained customers with citywise sales
citywise_kpi = citywise_sales.join(retained_customers_city, "StoreLocation", "left") \
    .fillna(0)

# Step 6: Calculate Retention Rate
citywise_kpi = citywise_kpi.withColumn("RetentionRate", (col("RetainedCustomers") / col("UniqueCustomers")) * 100)

# Step 7: Find Top Selling Product Category by City
top_product_category = df_with_date.groupBy("StoreLocation", "ProductCategory") \
    .agg(F.sum("TotalAmount").alias("CategorySales")) \
    .withColumn("Rank", F.row_number().over(Window.partitionBy("StoreLocation").orderBy(col("CategorySales").desc()))) \
    .filter(col("Rank") == 1) \
    .select("StoreLocation", "ProductCategory", "CategorySales")

# Step 8: Join Top Selling Product Category with Citywise KPI
df_final_city_kpi_test=citywise_kpi.join(top_product_category, "StoreLocation", "left") \
    .withColumnRenamed("ProductCategory", "TopSellingProductCategory") \
    .withColumnRenamed("CategorySales", "TopSellingCategorySales")\
    .orderBy("CategorySales",ascending=True)

# Show final KPI results
df_final_city_kpi_test.select("StoreLocation","TotalSales","TotalTransactions","AvgTransactionValue", 
                 "UniqueCustomers","RetainedCustomers","RetentionRate", 
                 "TopSellingProductCategory","TopSellingCategorySales")

display(df_final_city_kpi_test)

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
Phoenix,189.64,1,1,189.64,0,0.0,Bone Health (OTC & Prescription),189.64
New York City,238.14,1,1,238.14,0,0.0,Cardiovascular Diseases (Prescription),238.14
San Jose,434.28,2,2,217.14,0,0.0,Eye Health (OTC & Prescription),284.28
Chicago,487.8,1,1,487.8,0,0.0,Antibiotics (Prescription),487.8
Philadelphia,813.2,1,1,813.2,0,0.0,Diabetes (Prescription & OTC),813.2


In [0]:
prev_city_data_kpi_df_test2=spark.table("project_adb.postest.final_city_kpi_test3")
display(prev_city_data_kpi_df_test2)

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
Chicago,0.0,0,0,0.0,0,0.0,null,0.0


In [0]:
from delta.tables import DeltaTable

In [0]:
# Load the existing non-Delta table into a DataFrame
df_non_delta2 = spark.table("project_adb.postest.final_city_kpi_test3")

# Overwrite the existing table with Delta format
df_non_delta2.write.format("delta").mode("overwrite").saveAsTable("project_adb.postest.final_city_kpi_test3")


In [0]:
from delta.tables import *
from pyspark.sql import functions as F

# Assuming df_final_city_kpi_test is your DataFrame containing the new KPI metrics

# Initialize the target Delta table
target_delta_table = DeltaTable.forName(spark, "project_adb.postest.final_city_kpi_test3")

# Perform the merge operation
target_delta_table.alias("target").merge(
    source=df_final_city_kpi_test.alias("source"),
    condition="target.StoreLocation = source.StoreLocation"  # Match on StoreLocation
).whenMatchedUpdate(
    set={
        "TotalSales": "COALESCE(target.TotalSales, 0) + COALESCE(source.TotalSales, 0)",
        "TotalTransactions": "COALESCE(target.TotalTransactions, 0) + COALESCE(source.TotalTransactions, 0)",
        "AvgTransactionValue": (
            "CASE WHEN COALESCE(target.TotalTransactions, 0) + COALESCE(source.TotalTransactions, 0) > 0 "
            "THEN (COALESCE(target.TotalSales, 0) + COALESCE(source.TotalSales, 0)) / "
            "(COALESCE(target.TotalTransactions, 0) + COALESCE(source.TotalTransactions, 0)) "
            "ELSE 0 END"
        ),
        "UniqueCustomers": "source.UniqueCustomers",
        "RetainedCustomers": "source.RetainedCustomers",
        "RetentionRate": "source.RetentionRate",
        "TopSellingProductCategory": "source.TopSellingProductCategory",  # Keep existing value
        "TopSellingCategorySales": "source.TopSellingCategorySales"
    }
).whenNotMatchedInsert(
    values={
        "StoreLocation": "source.StoreLocation",
        "TotalSales": "source.TotalSales",
        "TotalTransactions": "source.TotalTransactions",
        "AvgTransactionValue": "source.AvgTransactionValue",
        "UniqueCustomers": "source.UniqueCustomers",
        "RetainedCustomers": "source.RetainedCustomers",
        "RetentionRate": "source.RetentionRate",
        "TopSellingProductCategory": "source.TopSellingProductCategory",
        "TopSellingCategorySales": "source.TopSellingCategorySales"
    }
).execute()

# If the DataFrame is empty and you want to ensure that something is inserted, consider adding a default row
if df_final_city_kpi_test.count() == 0:
    # Define a default row if no data exists
    default_row = spark.createDataFrame([{
        "StoreLocation": "Default Location",
        "TotalSales": 0,
        "TotalTransactions": 0,
        "AvgTransactionValue": 0,
        "UniqueCustomers": 0,
        "RetainedCustomers": 0,
        "RetentionRate": 0,
        "TopSellingProductCategory": "None",
        "TopSellingCategorySales": 0
    }])

    # Insert the default row into the Delta table
    default_row.write.format("delta").mode("append").saveAsTable("project_adb.postest.final_city_kpi_test2")


In [0]:
%sql
SELECT * FROM project_adb.postest.final_city_kpi_test3

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
Phoenix,189.64,1,1,189.64,0,0.0,Bone Health (OTC & Prescription),189.64
New York City,238.14,1,1,238.14,0,0.0,Cardiovascular Diseases (Prescription),238.14
San Jose,434.28,2,2,217.14,0,0.0,Eye Health (OTC & Prescription),284.28
Chicago,487.8,1,1,487.8,0,0.0,Antibiotics (Prescription),487.8
Philadelphia,813.2,1,1,813.2,0,0.0,Diabetes (Prescription & OTC),813.2


In [0]:
%sql
SELECT count(*) AS UPDATED_COUNT FROM project_adb.postest.final_city_kpi_test3

UPDATED_COUNT
5


In [0]:
#Save output file in different file formats with name like 'part-00000' in adls

df2=spark.table("project_adb.postest.final_city_kpi_test3")
df2.write.mode("overwrite").format("csv").save("/mnt/pos_data_testing/df_city_filtered.csv",header=True)

In [0]:
from pyspark.sql.functions import current_date

# Get the current date in 'yyyy-MM-dd' format
current_date_value = spark.sql("SELECT date_format(current_date(), 'yyyy-MM-dd')").collect()[0][0]
print(current_date_value)

# Create the directory for today's date
dbutils.fs.mkdirs(f"dbfs:/mnt/pos_data_testing/test-date-processed-output-city/{current_date_value}")

# Define the destination path for saving the DataFrame
destination_path_csv = f"dbfs:/mnt/pos_data_testing/test-date-processed-output-city/{current_date_value}/output_file"

# Write the DataFrame to CSV
df2.write.mode("overwrite").format("csv").save(destination_path_csv, header=True)

# List files in the source directory
files = dbutils.fs.ls("dbfs:/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/")

# Define custom file name for the copied file
custom_file_name = f"city_kpi_{current_date_value}.csv"

# Copy the relevant file(s) to the destination directory with the custom name
for file in files:
    if "part-00000-tid-" in file.name:
        # Copy the file to the destination with the custom name
        dbutils.fs.cp(file.path, f"dbfs:/mnt/pos_data_testing/test-date-processed-output-city/{current_date_value}/{custom_file_name}")
        break

# Now, move the entire output directory to a new name
new_directory_path = f"dbfs:/mnt/pos_data_testing/test-date-processed-output-city/{current_date_value}/output_file_temp"
dbutils.fs.mv(destination_path_csv, new_directory_path, recurse=True)

# Rename part files within the new directory to the custom file name
part_files = dbutils.fs.ls(new_directory_path)

# Move part files to the new custom name
for part_file in part_files:
    if part_file.name.startswith("part-"):
        dbutils.fs.mv(part_file.path, f"dbfs:/mnt/pos_data_testing/test-date-processed-output-city/{current_date_value}/{custom_file_name}",recurse=True)


2024-09-26


In [0]:
# %sql
# TRUNCATE TABLE project_adb.postest.final_city_kpi_test3